In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pyarrow
from statistics import median
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('../src/data/processed/data_initial.csv', index_col='SK_ID_CURR')

In [3]:
df.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,0,0,0,351000.0,0.018801,-9461,-637.0,-3648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,0.0,1,0,1,0,1129500.0,0.003541,-16765,-1188.0,-1186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,0.0,0,1,0,0,135000.0,0.010032,-19046,-225.0,-4260.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,0.0,1,0,0,0,297000.0,0.008019,-19005,-3039.0,-9833.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,6.0
100007,0.0,0,0,0,0,513000.0,0.028663,-19932,-3038.0,-4311.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.dropna(subset=['TARGET'], inplace=True)

In [5]:
# On supprime le genre du df, c'est toxique et n'a aucun sens
del df['CODE_GENDER']

In [6]:
df.describe()

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
count,307507.00000,307507.000000,307507.000000,307507.000000,3.072290e+05,307507.000000,307507.000000,252133.000000,307507.000000,307507.000000,...,86905.000000,86905.000000,86273.000000,86905.000000,86905.000000,86273.00000,86905.0,86905.0,86273.0,86905.000000
mean,0.08073,0.340106,0.306331,0.417047,5.383977e+05,0.020868,-16037.027271,-2384.142254,-4986.131376,-2994.201670,...,0.000064,0.005293,0.000064,0.003738,0.114366,0.00307,0.0,0.0,0.0,37.143605
std,0.27242,0.473745,0.460970,0.722119,3.694472e+05,0.013831,4363.982424,2338.327666,3522.883030,1509.454566,...,0.000877,0.072562,0.000881,0.031882,1.323208,0.02036,0.0,0.0,0.0,33.698512
min,0.00000,0.000000,0.000000,0.000000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,1.000000
25%,0.00000,0.000000,0.000000,0.000000,2.385000e+05,0.010006,-19682.000000,-3175.000000,-7479.500000,-4299.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,10.000000
50%,0.00000,0.000000,0.000000,0.000000,4.500000e+05,0.018850,-15750.000000,-1648.000000,-4504.000000,-3254.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,22.000000
75%,0.00000,1.000000,1.000000,1.000000,6.795000e+05,0.028663,-12413.000000,-767.000000,-2010.000000,-1720.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,75.000000
max,1.00000,1.000000,1.000000,19.000000,4.050000e+06,0.072508,-7489.000000,0.000000,0.000000,0.000000,...,0.024390,1.000000,0.024390,1.000000,62.000000,0.50000,0.0,0.0,0.0,192.000000


In [7]:
df.columns

Index(['TARGET', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MAX',
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT'],
      dtype='object', length=790)

In [8]:
df.dtypes

TARGET                              float64
FLAG_OWN_CAR                          int64
FLAG_OWN_REALTY                       int64
CNT_CHILDREN                          int64
AMT_GOODS_PRICE                     float64
                                     ...   
CC_NAME_CONTRACT_STATUS_nan_MAX      object
CC_NAME_CONTRACT_STATUS_nan_MEAN    float64
CC_NAME_CONTRACT_STATUS_nan_SUM     float64
CC_NAME_CONTRACT_STATUS_nan_VAR     float64
CC_COUNT                            float64
Length: 790, dtype: object

In [9]:
df.describe()

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
count,307507.00000,307507.000000,307507.000000,307507.000000,3.072290e+05,307507.000000,307507.000000,252133.000000,307507.000000,307507.000000,...,86905.000000,86905.000000,86273.000000,86905.000000,86905.000000,86273.00000,86905.0,86905.0,86273.0,86905.000000
mean,0.08073,0.340106,0.306331,0.417047,5.383977e+05,0.020868,-16037.027271,-2384.142254,-4986.131376,-2994.201670,...,0.000064,0.005293,0.000064,0.003738,0.114366,0.00307,0.0,0.0,0.0,37.143605
std,0.27242,0.473745,0.460970,0.722119,3.694472e+05,0.013831,4363.982424,2338.327666,3522.883030,1509.454566,...,0.000877,0.072562,0.000881,0.031882,1.323208,0.02036,0.0,0.0,0.0,33.698512
min,0.00000,0.000000,0.000000,0.000000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,1.000000
25%,0.00000,0.000000,0.000000,0.000000,2.385000e+05,0.010006,-19682.000000,-3175.000000,-7479.500000,-4299.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,10.000000
50%,0.00000,0.000000,0.000000,0.000000,4.500000e+05,0.018850,-15750.000000,-1648.000000,-4504.000000,-3254.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,22.000000
75%,0.00000,1.000000,1.000000,1.000000,6.795000e+05,0.028663,-12413.000000,-767.000000,-2010.000000,-1720.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,75.000000
max,1.00000,1.000000,1.000000,19.000000,4.050000e+06,0.072508,-7489.000000,0.000000,0.000000,0.000000,...,0.024390,1.000000,0.024390,1.000000,62.000000,0.50000,0.0,0.0,0.0,192.000000


# 1-GESTION DES NA ET DROP

In [10]:
colonnes_a_traiter = [
    'BURO_DAYS_CREDIT_MIN', 'BURO_DAYS_CREDIT_MAX', 'BURO_DAYS_CREDIT_MEAN', 'BURO_DAYS_CREDIT_VAR',
    'BURO_AMT_CREDIT_SUM_DEBT_MAX', 'BURO_AMT_CREDIT_SUM_DEBT_MEAN', 'BURO_AMT_CREDIT_SUM_DEBT_SUM',
    'BURO_AMT_CREDIT_SUM_OVERDUE_MEAN', 'BURO_AMT_CREDIT_SUM_LIMIT_MEAN', 'BURO_AMT_CREDIT_SUM_LIMIT_SUM',
    'BURO_AMT_ANNUITY_MAX', 'BURO_AMT_ANNUITY_MEAN', 'BURO_CNT_CREDIT_PROLONG_SUM',
    'BURO_MONTHS_BALANCE_MIN_MIN', 'BURO_MONTHS_BALANCE_MAX_MAX', 'BURO_MONTHS_BALANCE_SIZE_MEAN',
    'BURO_MONTHS_BALANCE_SIZE_SUM', 'BURO_CREDIT_ACTIVE_Active_MEAN', 'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
    'BURO_CREDIT_ACTIVE_Closed_MEAN', 'BURO_CREDIT_ACTIVE_Sold_MEAN', 'BURO_CREDIT_ACTIVE_nan_MEAN',
    'BURO_CREDIT_CURRENCY_currency 1_MEAN', 'BURO_CREDIT_CURRENCY_currency 2_MEAN',
    'BURO_CREDIT_CURRENCY_currency 3_MEAN', 'BURO_CREDIT_CURRENCY_currency 4_MEAN',
    'BURO_CREDIT_CURRENCY_nan_MEAN', 'BURO_CREDIT_TYPE_Another type of loan_MEAN',
    'BURO_CREDIT_TYPE_Car loan_MEAN', 'BURO_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN',
    'BURO_CREDIT_TYPE_Consumer credit_MEAN', 'BURO_CREDIT_TYPE_Credit card_MEAN',
    'BURO_CREDIT_TYPE_Interbank credit_MEAN', 'BURO_CREDIT_TYPE_Loan for business development_MEAN',
    'BURO_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN',
    'BURO_CREDIT_TYPE_Loan for the purchase of equipment_MEAN',
    'BURO_CREDIT_TYPE_Loan for working capital replenishment_MEAN',
    'BURO_CREDIT_TYPE_Microloan_MEAN', 'BURO_CREDIT_TYPE_Mobile operator loan_MEAN',
    'BURO_CREDIT_TYPE_Mortgage_MEAN', 'BURO_CREDIT_TYPE_Real estate loan_MEAN',
    'BURO_CREDIT_TYPE_Unknown type of loan_MEAN', 'BURO_CREDIT_TYPE_nan_MEAN',
    'BURO_STATUS_0_MEAN_MEAN', 'BURO_STATUS_1_MEAN_MEAN', 'BURO_STATUS_2_MEAN_MEAN',
    'BURO_STATUS_3_MEAN_MEAN', 'BURO_STATUS_4_MEAN_MEAN', 'BURO_STATUS_5_MEAN_MEAN',
    'BURO_STATUS_C_MEAN_MEAN', 'BURO_STATUS_X_MEAN_MEAN', 'BURO_STATUS_nan_MEAN_MEAN',
    'ACTIVE_DAYS_CREDIT_MIN', 'ACTIVE_DAYS_CREDIT_MAX', 'ACTIVE_DAYS_CREDIT_MEAN', 'ACTIVE_DAYS_CREDIT_VAR',
    'ACTIVE_DAYS_CREDIT_ENDDATE_MIN', 'ACTIVE_DAYS_CREDIT_ENDDATE_MAX', 'ACTIVE_DAYS_CREDIT_ENDDATE_MEAN',
    'ACTIVE_DAYS_CREDIT_UPDATE_MEAN', 'ACTIVE_CREDIT_DAY_OVERDUE_MAX', 'ACTIVE_CREDIT_DAY_OVERDUE_MEAN',
    'ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN', 'ACTIVE_AMT_CREDIT_SUM_MAX', 'ACTIVE_AMT_CREDIT_SUM_MEAN',
    'ACTIVE_AMT_CREDIT_SUM_SUM', 'ACTIVE_AMT_CREDIT_SUM_DEBT_MAX', 'ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN',
    'ACTIVE_AMT_CREDIT_SUM_DEBT_SUM', 'ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN', 'ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN',
    'ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM', 'ACTIVE_AMT_ANNUITY_MAX', 'ACTIVE_AMT_ANNUITY_MEAN',
    'ACTIVE_CNT_CREDIT_PROLONG_SUM', 'ACTIVE_MONTHS_BALANCE_MIN_MIN', 'ACTIVE_MONTHS_BALANCE_MAX_MAX',
    'ACTIVE_MONTHS_BALANCE_SIZE_MEAN', 'ACTIVE_MONTHS_BALANCE_SIZE_SUM',
    'CLOSED_DAYS_CREDIT_MIN', 'CLOSED_DAYS_CREDIT_MAX', 'CLOSED_DAYS_CREDIT_MEAN', 'CLOSED_DAYS_CREDIT_VAR',
    'CLOSED_DAYS_CREDIT_ENDDATE_MIN', 'CLOSED_DAYS_CREDIT_ENDDATE_MAX', 'CLOSED_DAYS_CREDIT_ENDDATE_MEAN',
    'CLOSED_DAYS_CREDIT_UPDATE_MEAN', 'CLOSED_CREDIT_DAY_OVERDUE_MAX', 'CLOSED_CREDIT_DAY_OVERDUE_MEAN',
    'CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN', 'CLOSED_AMT_CREDIT_SUM_MAX', 'CLOSED_AMT_CREDIT_SUM_MEAN',
    'CLOSED_AMT_CREDIT_SUM_SUM', 'CLOSED_AMT_CREDIT_SUM_DEBT_MAX', 'CLOSED_AMT_CREDIT_SUM_DEBT_MEAN',
    'CLOSED_AMT_CREDIT_SUM_DEBT_SUM', 'CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN', 'CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN',
    'CLOSED_AMT_CREDIT_SUM_LIMIT_SUM', 'CLOSED_AMT_ANNUITY_MAX', 'CLOSED_AMT_ANNUITY_MEAN',
    'CLOSED_CNT_CREDIT_PROLONG_SUM', 'CLOSED_MONTHS_BALANCE_MIN_MIN', 'CLOSED_MONTHS_BALANCE_MAX_MAX',
    'CLOSED_MONTHS_BALANCE_SIZE_MEAN', 'CLOSED_MONTHS_BALANCE_SIZE_SUM'
]
df = df.drop(columns=colonnes_a_traiter, axis = 1)
    

In [11]:
# Concernant les batiments, on ne garde que:
# TOTALAREA_MODE, LIVINGAREA_AVG, LIVINGAREA_MODE
df = df.drop([
                    'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
                    'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
                    'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
                    'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
                    'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
                    'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE',
                    'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE',
                    'LANDAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
                    'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
                    'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
                    'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
                    'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
                    'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MEDI',
                    'PREV_APP_CREDIT_PERC_MAX', 'PREV_APP_CREDIT_PERC_MEAN',
                    'REFUSED_APP_CREDIT_PERC_MAX', 'REFUSED_APP_CREDIT_PERC_MEAN',
                    'INSTAL_PAYMENT_PERC_MAX', 'INSTAL_PAYMENT_PERC_MEAN',
                    'INSTAL_PAYMENT_PERC_SUM'],       
    axis=1)

In [12]:
df.describe()

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
count,307507.00000,307507.000000,307507.000000,307507.000000,3.072290e+05,307507.000000,307507.000000,252133.000000,307507.000000,307507.000000,...,86905.000000,86905.000000,86273.000000,86905.000000,86905.000000,86273.00000,86905.0,86905.0,86273.0,86905.000000
mean,0.08073,0.340106,0.306331,0.417047,5.383977e+05,0.020868,-16037.027271,-2384.142254,-4986.131376,-2994.201670,...,0.000064,0.005293,0.000064,0.003738,0.114366,0.00307,0.0,0.0,0.0,37.143605
std,0.27242,0.473745,0.460970,0.722119,3.694472e+05,0.013831,4363.982424,2338.327666,3522.883030,1509.454566,...,0.000877,0.072562,0.000881,0.031882,1.323208,0.02036,0.0,0.0,0.0,33.698512
min,0.00000,0.000000,0.000000,0.000000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,1.000000
25%,0.00000,0.000000,0.000000,0.000000,2.385000e+05,0.010006,-19682.000000,-3175.000000,-7479.500000,-4299.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,10.000000
50%,0.00000,0.000000,0.000000,0.000000,4.500000e+05,0.018850,-15750.000000,-1648.000000,-4504.000000,-3254.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,22.000000
75%,0.00000,1.000000,1.000000,1.000000,6.795000e+05,0.028663,-12413.000000,-767.000000,-2010.000000,-1720.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,75.000000
max,1.00000,1.000000,1.000000,19.000000,4.050000e+06,0.072508,-7489.000000,0.000000,0.000000,0.000000,...,0.024390,1.000000,0.024390,1.000000,62.000000,0.50000,0.0,0.0,0.0,192.000000


In [13]:

df['AMT_GOODS_PRICE'].fillna(df['AMT_GOODS_PRICE'].median(), inplace=True)
df['DAYS_EMPLOYED'].fillna(0, inplace=True)
df['OWN_CAR_AGE'].fillna(df['OWN_CAR_AGE'].median(), inplace=True)
df['CNT_FAM_MEMBERS'].fillna(0, inplace=True)
df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].median(), inplace=True)
df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].median(), inplace=True)
df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(df['AMT_REQ_CREDIT_BUREAU_HOUR'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(df['AMT_REQ_CREDIT_BUREAU_DAY'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(df['AMT_REQ_CREDIT_BUREAU_WEEK'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_MON'].fillna(df['AMT_REQ_CREDIT_BUREAU_MON'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(df['AMT_REQ_CREDIT_BUREAU_QRT'].median(), inplace=True)
df['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(df['AMT_REQ_CREDIT_BUREAU_YEAR'].median(), inplace=True)

C:\Users\Alexa\AppData\Local\Temp\ipykernel_20908\1938375175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AMT_GOODS_PRICE'].fillna(df['AMT_GOODS_PRICE'].median(), inplace=True)
C:\Users\Alexa\AppData\Local\Temp\ipykernel_20908\1938375175.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [14]:
df.isna().sum()

TARGET                                   0
FLAG_OWN_CAR                             0
FLAG_OWN_REALTY                          0
CNT_CHILDREN                             0
AMT_GOODS_PRICE                          0
                                     ...  
CC_NAME_CONTRACT_STATUS_nan_MAX     220602
CC_NAME_CONTRACT_STATUS_nan_MEAN    220602
CC_NAME_CONTRACT_STATUS_nan_SUM     220602
CC_NAME_CONTRACT_STATUS_nan_VAR     221234
CC_COUNT                            220602
Length: 637, dtype: int64

In [15]:
df.fillna(df['LIVINGAREA_AVG'].median(), inplace=True)
df.fillna(df['LIVINGAREA_MODE'].median(), inplace=True)
df.fillna(df['TOTALAREA_MODE'].median(), inplace=True)

In [16]:
df['OBS_30_CNT_SOCIAL_CIRCLE'].dropna(inplace=True)

In [17]:
df.isna().sum()

TARGET                              0
FLAG_OWN_CAR                        0
FLAG_OWN_REALTY                     0
CNT_CHILDREN                        0
AMT_GOODS_PRICE                     0
                                   ..
CC_NAME_CONTRACT_STATUS_nan_MAX     0
CC_NAME_CONTRACT_STATUS_nan_MEAN    0
CC_NAME_CONTRACT_STATUS_nan_SUM     0
CC_NAME_CONTRACT_STATUS_nan_VAR     0
CC_COUNT                            0
Length: 637, dtype: int64

In [18]:
df.isna().sum().sum()

0

In [19]:
colonnes_object = df.select_dtypes(include=['object']).columns
df[colonnes_object]

,CC_NAME_CONTRACT_STATUS_Active_MIN,CC_NAME_CONTRACT_STATUS_Active_MAX,CC_NAME_CONTRACT_STATUS_Approved_MIN,CC_NAME_CONTRACT_STATUS_Approved_MAX,CC_NAME_CONTRACT_STATUS_Completed_MIN,CC_NAME_CONTRACT_STATUS_Completed_MAX,CC_NAME_CONTRACT_STATUS_Demand_MIN,CC_NAME_CONTRACT_STATUS_Demand_MAX,CC_NAME_CONTRACT_STATUS_Refused_MIN,CC_NAME_CONTRACT_STATUS_Refused_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MIN,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX
SK_ID_CURR,,,,,,,,,,,,,,,,
100002,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
100003,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
100004,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
100006,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
100007,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
456252,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745
456253,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745,0.0745


In [20]:
valeurs_uniques = set()

# Parcourir chaque colonne et ajouter ses valeurs uniques à l'ensemble
for col in colonnes_object:
    valeurs_uniques.update(df[col].unique())

# Convertir l'ensemble en liste pour l'affichage
liste_valeurs_uniques = list(valeurs_uniques)
liste_valeurs_uniques

[0.0745, True, False]

In [21]:
df = df.drop(colonnes_object, axis=1)

# 2-CONVERTIR LES COLONNES POUR OPTIMISATION

In [22]:
int_col = df.select_dtypes(include=['int64']).columns
int_col

Index(['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'DAYS_BIRTH',
       'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
       'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
       'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21'],
      dtype='object')

In [23]:
min_max_values = df[int_col].agg(['min', 'max'])

# Afficher les résultats
min_max_values

,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
min,0,0,0,-25229,-7197,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
max,1,1,19,-7489,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [24]:
# Pour chaque colonne dans int_col, afficher ses valeurs uniques
for col in int_col:
    unique_values = df[col].unique()
    print(f"\nColonne: {col}")
    print(f"Nombre de valeurs uniques: {len(unique_values)}")
    print("Valeurs uniques:")
    print(unique_values)
    print("-" * 50)


Colonne: FLAG_OWN_CAR
Nombre de valeurs uniques: 2
Valeurs uniques:
[0 1]
--------------------------------------------------

Colonne: FLAG_OWN_REALTY
Nombre de valeurs uniques: 2
Valeurs uniques:
[0 1]
--------------------------------------------------

Colonne: CNT_CHILDREN
Nombre de valeurs uniques: 15
Valeurs uniques:
[ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]
--------------------------------------------------

Colonne: DAYS_BIRTH
Nombre de valeurs uniques: 17460
Valeurs uniques:
[ -9461 -16765 -19046 ...  -7951  -7857 -25061]
--------------------------------------------------

Colonne: DAYS_ID_PUBLISH
Nombre de valeurs uniques: 6168
Valeurs uniques:
[-2120  -291 -2531 ... -6194 -5854 -6211]
--------------------------------------------------

Colonne: FLAG_MOBIL
Nombre de valeurs uniques: 2
Valeurs uniques:
[1 0]
--------------------------------------------------

Colonne: FLAG_EMP_PHONE
Nombre de valeurs uniques: 2
Valeurs uniques:
[1 0]
--------------------------------------

In [25]:
# Identifier les colonnes qui ne contiennent que 0 et 1
colonnes_binaires = []
for col in int_col:
    valeurs_uniques = set(df[col].unique())
    if valeurs_uniques.issubset({0, 1}):
        colonnes_binaires.append(col)

# Afficher les colonnes identifiées
print("Colonnes contenant uniquement 0 et 1:")
print(colonnes_binaires)

# Convertir ces colonnes en type booléen
df[colonnes_binaires] = df[colonnes_binaires].astype('bool')

# Vérifier le nouveau type de ces colonnes
print("\nNouveaux types des colonnes converties:")
print(df[colonnes_binaires].dtypes)

# Optionnel : afficher quelques lignes pour vérifier la conversion
print("\nAperçu des données converties:")
print(df[colonnes_binaires].head())

Colonnes contenant uniquement 0 et 1:
['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

Nouveaux types des colonnes converties:
FLAG_OWN_CAR                   bool
FLAG_OWN_REALTY                bool
FLAG_MOBIL                     bool
FLAG_EMP_PHONE                 bool
FLAG_WORK_PHONE                bool
FLAG_CONT_MOBILE               bool
FLAG_PHO

In [26]:
int_col2 = df.select_dtypes(include=['int64']).columns
int_col2

Index(['CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START'],
      dtype='object')

In [27]:
min_max_values2 = df[int_col2].agg(['min', 'max'])

# Afficher les résultats
min_max_values2

,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START
min,0,-25229,-7197,1,1,0
max,19,-7489,0,3,3,23


In [28]:
df['CNT_CHILDREN'] = df['CNT_CHILDREN'].astype('int8')
df['DAYS_BIRTH'] = df['DAYS_BIRTH'].astype('int16')
df['DAYS_ID_PUBLISH'] = df['DAYS_ID_PUBLISH'].astype('int16')
df['REGION_RATING_CLIENT'] = df['REGION_RATING_CLIENT'].astype('int8')
df['REGION_RATING_CLIENT_W_CITY'] = df['REGION_RATING_CLIENT_W_CITY'].astype('int8')
df['HOUR_APPR_PROCESS_START'] = df['HOUR_APPR_PROCESS_START'].astype('int8')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 100002 to 456255
Columns: 621 entries, TARGET to CC_COUNT
dtypes: bool(167), float64(448), int16(2), int8(4)
memory usage: 1.1 GB


In [30]:
# On supprime les colonnes en double, en gardant la première colonne
def remove_duplicate_columns(df):
    # Convertir le DataFrame en un tableau numpy pour un traitement plus rapide
    arr = df.values
    
    # Calculer un hash pour chaque colonne
    column_hashes = np.apply_along_axis(lambda x: hash(tuple(x)), axis=0, arr=arr)
    
    # Trouver les indices uniques
    _, unique_column_indices = np.unique(column_hashes, return_index=True)
    
    # Trier les indices pour préserver l'ordre original des colonnes
    unique_column_indices = np.sort(unique_column_indices)
    
    # Créer un nouveau DataFrame avec seulement les colonnes uniques
    df_unique = df.iloc[:, unique_column_indices]
    
    return df_unique

# Utilisation
df= remove_duplicate_columns(df)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 100002 to 456255
Columns: 601 entries, TARGET to CC_COUNT
dtypes: bool(167), float64(428), int16(2), int8(4)
memory usage: 1.0 GB


# 3- DESEQUILIBRE DES CLASSES

In [33]:
from imblearn.over_sampling import ADASYN

# Split
y = df['TARGET']
X = df.drop('TARGET', axis=1)

# Application d'ADASYN
ada = ADASYN(random_state=42)  # pour la reproductibilité
X_resampled, y_resampled = ada.fit_resample(X, y)

# Création du nouveau df équilibré
df_balanced = pd.DataFrame(X_resampled, columns=X.columns)
df_balanced['TARGET'] = y_resampled

C:\Users\Alexa\AppData\Local\Temp\ipykernel_20908\4006393345.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced['TARGET'] = y_resampled


# 4- SPLIT, FEATURES and OUTLIERS

In [91]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

In [92]:
print(y.dtype)
print(y.nunique)

float64
<bound method IndexOpsMixin.nunique of SK_ID_CURR
100002    1.0
100003    0.0
100004    0.0
100006    0.0
100007    0.0
         ... 
456251    0.0
456252    0.0
456253    0.0
456254    1.0
456255    0.0
Name: TARGET, Length: 307507, dtype: float64>


In [121]:
# Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)


In [122]:
top_features = feature_importance['feature'][:20].tolist()
top_features = top_features

In [123]:
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X, y)

# Calculer l'importance des features
feature_importance = pd.DataFrame({
    'feature_xgb': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Afficher les 20 features les plus importantes
print(feature_importance.head(20))

                                           feature_xgb  importance
98                                        EXT_SOURCE_3    0.037539
74                                        EXT_SOURCE_2    0.035687
167               NAME_EDUCATION_TYPE_Higher education    0.033372
43                            NAME_INCOME_TYPE_Working    0.015014
23                                     FLAG_DOCUMENT_3    0.013380
138                                 CC_AMT_BALANCE_MIN    0.013226
1                                         EXT_SOURCE_1    0.011714
0               PREV_NAME_CONTRACT_STATUS_Refused_MEAN    0.011680
111  NAME_EDUCATION_TYPE_Secondary / secondary special    0.011445
92                       APPROVED_AMT_DOWN_PAYMENT_MAX    0.011238
24                             OCCUPATION_TYPE_Drivers    0.011059
39                     CC_CNT_DRAWINGS_ATM_CURRENT_VAR    0.010905
161                   CC_CNT_DRAWINGS_ATM_CURRENT_MEAN    0.010422
145                             INSTAL_AMT_PAYMENT_SUM    0.01

In [124]:
top_features_xgb = feature_importance['feature_xgb'][:20].tolist()
top_features_xgb

['EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_INCOME_TYPE_Working',
 'FLAG_DOCUMENT_3',
 'CC_AMT_BALANCE_MIN',
 'EXT_SOURCE_1',
 'PREV_NAME_CONTRACT_STATUS_Refused_MEAN',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'APPROVED_AMT_DOWN_PAYMENT_MAX',
 'OCCUPATION_TYPE_Drivers',
 'CC_CNT_DRAWINGS_ATM_CURRENT_VAR',
 'CC_CNT_DRAWINGS_ATM_CURRENT_MEAN',
 'INSTAL_AMT_PAYMENT_SUM',
 'AMT_GOODS_PRICE',
 'INSTAL_DPD_MEAN',
 'CC_CNT_DRAWINGS_CURRENT_MEAN',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'NAME_INCOME_TYPE_State servant',
 'REG_CITY_NOT_LIVE_CITY']

In [125]:
top_feat = list(set(top_features_xgb + top_features))
len(top_feat)

37

In [126]:
top_feat

['PREV_NAME_CONTRACT_STATUS_Refused_MEAN',
 'CC_CNT_DRAWINGS_ATM_CURRENT_VAR',
 'DAYS_EMPLOYED',
 'REGION_POPULATION_RELATIVE',
 'EXT_SOURCE_1',
 'DAYS_LAST_PHONE_CHANGE',
 'BURO_DAYS_CREDIT_UPDATE_MEAN',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'AMT_GOODS_PRICE',
 'DAYS_BIRTH',
 'NAME_INCOME_TYPE_Working',
 'NAME_INCOME_TYPE_State servant',
 'LOG_CREDIT',
 'LOG_ANNUITY',
 'BURO_AMT_CREDIT_SUM_SUM',
 'CC_AMT_BALANCE_MIN',
 'APPROVED_AMT_DOWN_PAYMENT_MAX',
 'REG_CITY_NOT_LIVE_CITY',
 'EXT_SOURCE_3',
 'BURO_DAYS_CREDIT_ENDDATE_MIN',
 'FLAG_DOCUMENT_3',
 'INSTAL_AMT_PAYMENT_SUM',
 'BURO_DAYS_CREDIT_ENDDATE_MAX',
 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
 'OCCUPATION_TYPE_Drivers',
 'DAYS_ID_PUBLISH',
 'INSTAL_DPD_MEAN',
 'INSTAL_DBD_MEAN',
 'INSTAL_AMT_INSTALMENT_MAX',
 'CC_CNT_DRAWINGS_ATM_CURRENT_MEAN',
 'INSTAL_DBD_SUM',
 'INSTAL_DAYS_ENTRY_PAYMENT_MEAN',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'CC_CNT_DRAWINGS_CURRENT_MEAN',
 'EXT_SOURCE_2',
 'NAME_EDUCATION_TYPE_Higher education',
 'DAYS

In [127]:
top_feat.append('TARGET')

In [128]:
df_cleaned = df[top_feat]

In [129]:
def detect_outliers(df):
    outliers = {}
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.01)
        Q3 = df[column].quantile(0.99)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[column] = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
    return outliers

outliers = detect_outliers(df_cleaned)

# Afficher le nombre d'outliers par colonne
for column, indices in outliers.items():
    if len(indices) == 0:
        continue
    else:
        print(f"{column}: {len(indices)} outliers")

CC_CNT_DRAWINGS_ATM_CURRENT_VAR: 742 outliers
BURO_DAYS_CREDIT_UPDATE_MEAN: 35 outliers
DEF_30_CNT_SOCIAL_CIRCLE: 14 outliers
LOG_ANNUITY: 12 outliers
BURO_AMT_CREDIT_SUM_SUM: 244 outliers
CC_AMT_BALANCE_MIN: 540 outliers
APPROVED_AMT_DOWN_PAYMENT_MAX: 449 outliers
BURO_DAYS_CREDIT_ENDDATE_MIN: 1950 outliers
INSTAL_AMT_PAYMENT_SUM: 14 outliers
BURO_DAYS_CREDIT_ENDDATE_MEAN: 4 outliers
INSTAL_DPD_MEAN: 1134 outliers
INSTAL_DBD_MEAN: 90 outliers
INSTAL_AMT_INSTALMENT_MAX: 11 outliers
CC_CNT_DRAWINGS_ATM_CURRENT_MEAN: 230 outliers
INSTAL_DBD_SUM: 83 outliers
CC_CNT_DRAWINGS_CURRENT_MEAN: 371 outliers


In [130]:
len(df_cleaned.index)

307507

In [131]:
# Winsorisation à 5%
def detect_outliers(df):
    outliers = {}
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.05)
        Q3 = df[column].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[column] = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
    return outliers

# Detect outliers
outliers = detect_outliers(df_cleaned)

outlier_indices = set()
for indices in outliers.values():
    outlier_indices.update(indices)

# Création du masque
mask = ~df_cleaned.index.isin(outlier_indices)

# Suppression des outliers
df_cleaned_without_outliers = df_cleaned[mask]

In [132]:
print(df_cleaned_without_outliers.shape)

(271616, 38)


In [133]:
df[top_feat].to_parquet("df_cleaned_without_outliers")